# Climate Data 

In [1]:
from gurobipy import *
import time
import pandas as pd

In [2]:
hw85 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp85\\hotdays.csv", delimiter=',' , header=None)
hr85 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp85\\mediumrain.csv", delimiter=',' , header=None)
sp85=  pd.read_csv( r"C:\Users\NLLIST\OneDrive - Sweco AB\Thesis\Climate data\rcp85\sunpeak.csv", delimiter=' ' , header=None)

hw45 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp45\\hotdays.csv", delimiter=',' , header=None)
hr45 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp45\\mediumrain.csv", delimiter=',' , header=None)
sp45 =  pd.read_csv( r"C:\Users\NLLIST\OneDrive - Sweco AB\Thesis\Climate data\rcp45\sunpeak.csv", delimiter=' ' , header=None)

hw26 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp26\\hotdays.csv", delimiter=',' , header=None)
hr26 = pd.read_csv( r"C:\\Users\\NLLIST\\OneDrive - Sweco AB\\Thesis\\Climate data\\rcp26\\mediumrain.csv", delimiter=',' , header=None)
sp26 =  pd.read_csv( r"C:\Users\NLLIST\OneDrive - Sweco AB\Thesis\Climate data\rcp26\sunpeak.csv", delimiter=' ' , header=None)

In [3]:
pand = pd.read_csv(r"C:\Users\NLLIST\Desktop\Thesis\try6\Merged.csv")

In [4]:
pand.describe()

,pandid,ig,cb,rg,rb,_area,ry,AREA
count,1.120000e+03,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,877.000000
mean,6.061000e+14,0.006192,265.491071,0.592735,0.421259,1168.907379,0.645361,727.776511
std,6.550399e+03,0.024403,105.876190,0.180262,0.142423,2097.225753,0.355074,1251.096492
min,6.061000e+14,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,30.000000
25%,6.061000e+14,0.000000,200.000000,0.455846,0.309640,235.808196,0.591281,181.750000
50%,6.061000e+14,0.000352,200.000000,0.564605,0.403950,574.317823,0.837655,322.500000
75%,6.061000e+14,0.003171,350.000000,0.712001,0.504103,1172.027171,0.888864,703.500000
max,6.061000e+14,0.371587,500.000000,1.000000,1.000000,32953.665197,0.963716,15385.500000


## Read data from pand

In [5]:


#Roofs
a=pand["_area"] # ROOFS' AREAS
roofs = range(len(a)) # SET OF ROOFS

#green
rg=pand["rg"] # HEAT MITIGATION POTENTIAL
ig=pand['ig'] 

#blue
rb=pand["rb"]

#yellow
ry = pand["ry"]

## Read and process climate-depending parameters

In [6]:


#blue
# AVOIDED COSTS PER M3 OF WATER RETAINED, FOR EACH RAINFALL EVENT
cb85=pand["cb"]/(1000*len(hr85)) 
cb45=pand["cb"]/(1000*len(hr45))
cb26=pand["cb"]/(1000*len(hr26))

r85=hr85[1]
r45=hr45[1]
r26=hr26[1]

#green

# we cosider heat can be mitigated thanks to the presence of green roofs only when the
# temperatures are between 25 (excluded) and 31 degrees. A higher temperature won't allow for any relief, 
# while a lower temprature is not considered as damaging

h85=hw85[1]
h45=hw45[1]
h26=hw26[1]

h85 = h85[(h85 >=25) & (h85 <=31)]
h85= h85.reset_index(drop=True) # indexing has to be updated in order to further process the vector

h45 = h45[(h45 >=25) & (h45 <=31)]
h45= h45.reset_index(drop=True)

h26= h26[(h26 >=25) & (h26 <=31)]
h26= h26.reset_index(drop=True)

w, r = len(h85), len(a); #create a matrix with dimensions roofs x heat events
m85 = [[0 for j in range(w)] for i in range(r)] 
for i in range(r):
    for j in range(w):
        m85[i][j]=ig[i]+(ig[i]/5)*(h85[j]-30)        
w, r = len(h45), len(a); #create a matrix with dimensions roofs x heat events
m45 = [[0 for j in range(w)] for i in range(r)] 
for i in range(r):
    for j in range(w):
        m45[i][j]=ig[i]+(ig[i]/5)*(h45[j]-30)          
w, r = len(h26), len(a); #create a matrix with dimensions roofs x heat events
m26 = [[0 for j in range(w)] for i in range(r)] 
for i in range(r):
    for j in range(w):
        m26[i][j]=ig[i]+(ig[i]/5)*(h26[j]-30) 

m85=pd.Series(m85).fillna(0).tolist()
m45=pd.Series(m45).fillna(0).tolist()
m26=pd.Series(m26).fillna(0).tolist()

#yellow
ds85=int(sp85.sum())
ds45=int(sp45.sum())
ds26=int(sp26.sum())



## User parameters

In [7]:
cg= 5000
kb=0.8
kg=0.8
Q=187.5
theta= 0.05
hg=10
cy=0.0700734/1000

# Choose the budget
B=5000000

# Choose the subsidy rate 
sub=0.2

# Prices per m2 of roof type
py=470 #YR
pb=50 #BR
pg=80 #GR

## Stochastic model

In [8]:
try:
    # Create a new model called m
    m = Model("multiple_roofs")
    # Set the model to maximize
    m.ModelSense = -1    

    ## Variables 
    xYR = m.addVars(len(a), name='yr')
    xGR = m.addVars(len(a), name='gr')
    xBR = m.addVars(len(a), name= 'br')

    yGR = m.addVars(len(a), vtype=GRB.BINARY,name='bin_gr')
    yBR = m.addVars(len(a), vtype=GRB.BINARY,name='bin_br')

    bYR = m.addVar(obj=1/3, name='benefit_yr')
    bGR = m.addVar(obj=1/3, name='benefit_gr')
    bBR = m.addVar(obj=1/3, name='benefit_br')


    ## Constraints
    #area
    m.addConstrs((xGR[i] == a[i]*yGR[i] for i in roofs),name='area_gr')
    m.addConstrs((xBR[i] == a[i]*yBR[i] for i in roofs),name='area_br')
    m.addConstrs((xYR[i] <= a[i] for i in roofs), name='area_yr')

    #budget
    m.addConstr( (quicksum(sub*(pg*xGR[i] + py*xYR[i]+ pb*xBR[i]) for i in roofs) <= B), name='budget') #

    #technical choices
    m.addConstrs(yGR[i]>=yBR[i] for i in roofs)
    #m.addConstrs(xYR[i]==0 for i in roofs)



    #benefits
    m.addConstr(bGR ==quicksum( quicksum(cg*m85[i][w] *rg[i] *yGR[i] for w in range(len(h85))) + quicksum(cg*m45[i][w] *rg[i] *yGR[i] for w in range(len(h45))) + quicksum(cg*m26[i][w] *rg[i] *yGR[i] for w in range(len(h26))) for i in roofs), name='benefit_gr')
    m.addConstr(bBR ==quicksum( quicksum(cb85[i]*(kb*r85[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for j in range(len(r85))  ) + quicksum(cb45[i]*(kb*r45[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for j in range(len(r45)) ) + quicksum(cb26[i]*(kb*r26[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for j in range(len(r26))) for i in roofs), name='benefit_br')
    m.addConstr(bYR == quicksum( cy* Q *ds85 * ry[i] *(1+theta*yGR[i])* xYR[i]  + cy* Q *ds45 * ry[i] *(1+theta*yGR[i])* xYR[i] + cy* Q *ds26 * ry[i] *(1+theta*yGR[i])* xYR[i] for i in roofs) , name='benefit_yr')

    m.update()
    m.optimize()

    # Here the solution is stored in such a way that the different combination of outcomes can be investigated


    i_yr = [i for i in roofs if xYR[i].X > 0]
    i_br = [i for i in roofs if yBR[i].X > 0]
    i_gr = [i for i in roofs if yGR[i].X > 0]

    pand.loc[i_yr, 'yr']=1
    pand.loc[i_br, 'br']=1
    pand.loc[i_gr, 'gr']=1
    
    

    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))

    print('Obj: %g' % m.objVal)
except GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')
    

Academic license - for non-commercial use only - expires 2021-09-16
Using license file C:\Users\NLLIST\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4483 rows, 5603 columns and 14199 nonzeros
Model fingerprint: 0x1134fc71
Model has 1 quadratic constraint
Variable types: 3363 continuous, 2240 integer (2240 binary)
Coefficient statistics:
  Matrix range     [7e-03, 5e+05]
  QMatrix range    [2e-01, 1e+01]
  QLMatrix range   [1e+00, 3e+02]
  Objective range  [3e-01, 3e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 5e+06]
Found heuristic solution: objective -0.0000000
Presolve removed 3366 rows and 2494 columns
Presolve time: 0.01s
Presolved: 1992 rows, 3984 columns, 7966 nonzeros
Found heuristic solution: objective 519.5683736
Variable types: 1750 continuous, 2234 integer (2234 binary)

Root relaxation: objective 5.335589e+06, 52 iterations, 0.00 seconds

# Robust model

In [15]:
####

# GIVEN A CERTAIN BUDGET, THE MUNICIPALITY WISHES TO PLACE GREEN, BLUE AND YELLOW ROOFS IN SUCH A 
# WAY THAT THE CITY BECOMES CLIMATE RESILIENT. FLOODING AND HEAT MITIGATION ARE CLIMATE ADAPTATION 
# GOALS, WHILE THE PRODUCTION OF CLEAN ENERGY IS A CLIMATE MITIGATION GOAL.

# GR: green roofs
# BR: blue roofs
# YR: yellow roofs

out=[]
j=0
# Time for implementing and solving the model is printed at the end of the script. it starts from here.
start_time = time.time()

try:

    # Create a new model called m
    m = Model("multiple_roofs") 
    m.ModelSense = -1 
    

    ## Variables 
    xYR = m.addVars(len(a), name='yr')
    xGR = m.addVars(len(a), name='gr')
    xBR = m.addVars(len(a), name= 'br')

    yGR = m.addVars(len(a), vtype=GRB.BINARY,name='bin_gr')
    yBR = m.addVars(len(a), vtype=GRB.BINARY,name='bin_br')
    worst = m.addVar(obj=1,name='worst')
    

    bYR85 = m.addVar( name='benefit_yr_85')
    bGR85 = m.addVar( name='benefit_gr_85')
    bBR85 = m.addVar( name='benefit_br_85')
    
    bYR45 = m.addVar(  name='benefit_yr_45')
    bGR45 = m.addVar(  name='benefit_gr_45')
    bBR45 = m.addVar(  name='benefit_br_45')
    
    bYR26 = m.addVar(  name='benefit_yr_26')
    bGR26 = m.addVar(  name='benefit_gr_26')
    bBR26 = m.addVar(  name='benefit_br_26')

    
    ## Constraints
    #area
    m.addConstrs((xGR[i] == a[i]*yGR[i] for i in roofs),name='area_gr')
    m.addConstrs((xBR[i] == a[i]*yBR[i] for i in roofs),name='area_br')
    m.addConstrs((xYR[i] <= a[i] for i in roofs), name='area_yr')

    #budget
    m.addConstr( (quicksum(sub*(pg*xGR[i] + py*xYR[i] +pb*xBR[i]) for i in roofs) <= B), name='budget') #

    #technical choices
    m.addConstrs(yGR[i]>=yBR[i] for i in roofs)
    #m.addConstrs(xYR[i]==0 for i in roofs)
    

    #benefits
    m.addConstr(bGR85== quicksum(cg*m85[i][w] *rg[i] *yGR[i] for i in roofs for w in range(len(h85))) )
    m.addConstr(bBR85 == quicksum(cb85[i]*(kb*r85[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for i in roofs for j in range(len(r85))) ) 
    m.addConstr(bYR85== quicksum(cy* Q *ds85 * ry[i] *(1+theta*yGR[i])* xYR[i] for i in roofs))

    m.addConstr(bGR45 == quicksum(cg*m45[i][w] *rg[i] *yGR[i] for i in roofs for w in range(len(h45))) )
    m.addConstr(bBR45 == quicksum(cb45[i]*(kb*r45[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for i in roofs for j in range(len(r45))))
    m.addConstr(bYR45 == quicksum(cy* Q *ds45 * ry[i] *(1+theta*yGR[i])* xYR[i] for i in roofs))

    m.addConstr(bGR26 == quicksum(cg*m26[i][w] *rg[i] *yGR[i] for i in roofs for w in range(len(h26)))  )
    m.addConstr(bBR26 == quicksum(cb26[i]*(kb*r26[j]*xBR[i]+kg*hg*xGR[i])*rb[i] for i in roofs for j in range(len(r26))) )
    m.addConstr(bYR26 == quicksum(cy* Q *ds26 * ry[i] *(1+theta*yGR[i])* xYR[i] for i in roofs))
    
    m.addConstr(worst<= bGR85+bBR85+bYR85)
    m.addConstr(worst<= bGR45+bBR45+bYR45)
    m.addConstr(worst<= bGR26+bBR26+bYR26)

    m.update()
    m.optimize()

    # Here the solution is stored in such a way that the different combination of outcomes can be investigated
    out.append(m.objVal)

    i_yr = [i for i in roofs if xYR[i].X > 0]
    i_br = [i for i in roofs if yBR[i].X > 0]
    i_gr = [i for i in roofs if yGR[i].X > 0]

    pand.loc[i_yr, 'yr_rob']=1
    pand.loc[i_br, 'br_rob']=1
    pand.loc[i_gr, 'gr_rob']=1



        #for v in m.getVars():
        #    print('%s %g' % (v.varName, v.x))

        #print('Obj: %g' % m.objVal)

except GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')
    
print('Time required to run the model:')
print("--- %s seconds ---" % (time.time() - start_time)) 
print('MB of memory allocated to run the model:')
import os, psutil; print(psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4490 rows, 5610 columns and 20217 nonzeros
Model fingerprint: 0xd2daddb3
Model has 3 quadratic constraints
Variable types: 3370 continuous, 2240 integer (2240 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+05]
  QMatrix range    [6e-02, 5e+00]
  QLMatrix range   [1e+00, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 5e+06]
Found heuristic solution: objective -0.0000000
Presolve removed 3368 rows and 2496 columns
Presolve time: 0.03s
Presolved: 3748 rows, 3989 columns, 20694 nonzeros
Variable types: 1754 continuous, 2235 integer (2235 binary)

Root relaxation: objective -5.257784e+06, 1486 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*   